In [ ]:
# Install dependencies
%pip install ultralytics
%pip install opencv-python
%pip install cv2-enumerate-cameras

In [ ]:
# Configurations
import cv2
import math
import time
from ultralytics import YOLO
from cv2_enumerate_cameras import enumerate_cameras

# Camera configuration
CAMERA_INDEX = 0                 # Index of the camera to be used (custom value; check with enumerate_cameras)
FRAME_DELAY = 0.2                # Delay between frames in seconds for processing

# YOLO model setup
MODEL = YOLO('yolov8n.pt')       # Load YOLOv8 nano model for object detection
CLASS_NAMES = MODEL.names        # List of all class labels in the model
DESIRED_CLASSES = ["balloon"]    # Classes we want to detect (only track balloons)

In [ ]:
# CV2 and camera settings

# Initialize camera
def init_camera(index=CAMERA_INDEX, verbose=True):
    cap = cv2.VideoCapture(index)
    if not cap.isOpened():
        raise RuntimeError(f"Error: Could not open webcam {index}.")
    if verbose:
        print(f"Camera {index} opened successfully.")
    return cap


# Read current frame
def read_frame(cap):
    ret, frame = cap.read()
    if not ret:
        raise RuntimeError("Failed to capture image from webcam.")
    return frame


# Calculate frame center
def calculate_center(frame):
    height, width = frame.shape[:2]
    x_center, y_center = width / 2, height / 2
    return x_center, y_center


# Check if class is in desired classes
def valid_class(box, class_names=CLASS_NAMES, desired_classes=DESIRED_CLASSES):
    return class_names[int(box.cls)] in desired_classes


# Draw bounding box
def draw_bounding_box(frame, box, class_names=CLASS_NAMES, verbose=True):
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)
    
    cls = int(box.cls[0])
    confidence = math.ceil((box.conf[0] * 100)) / 100
    
    org = [x1, y1]
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    color = (255, 0, 0)
    thickness = 2
    cv2.putText(frame, class_names[cls], org, font, font_scale, color, thickness)
    
    if verbose:
        print(f"Class: {class_names[cls]}, Confidence: {confidence}")

In [ ]:
# Print functions

# Print divider
def print_div(text):
    print("\n" + f" {text} ".center(100, '-') + "\n")


# Print available cameras
def print_cameras():
    for camera_info in enumerate_cameras():
        print(f'{camera_info.index}: {camera_info.name}')


# Print final information
def final_prints(frame_count, x_center, y_center):
    print(f"Total frames: {frame_count}")
    print(f"Detected classes: {DESIRED_CLASSES}")
    print(f"Frame center: ({x_center}, {y_center})")
    print(f"FPS: {(1/FRAME_DELAY):.2f}")

In [ ]:
# Main program
print_div("Available cameras")
print_cameras()

print_div("Initial settings")
frame_count = 0
cap = init_camera()
frame = read_frame(cap)
x_center, y_center = calculate_center(frame)

print_div("Program started")
try:
    while True:
        frame = read_frame(cap)
        results = MODEL(frame, stream=True)
        for r in results:
            if not r.boxes:
                continue
            for box in r.boxes:
                if valid_class(box):
                    draw_bounding_box(frame, box)
        
        cv2.imshow("Image", frame)
        if cv2.waitKey(1) == ord('q'):
            break
        
        print(f"Frame number: {frame_count}")
        frame_count += 1
        time.sleep(FRAME_DELAY)
finally:
    print_div("Program finished")
    final_prints(frame_count, x_center, y_center)
    cap.release()
    cv2.destroyAllWindows()